# Connecting a FABRIC slice to the National Research Platform (NRP) cluster at AtlanticWave/AmLight resource using a Layer 2 facility port

This notebook creates a slice where a virtual machine will run a iperf3 test against a NRP pod running at AmLight/AMPATH. 
To achieve such a goal, the slice must be expanded with a Layer 2 facility port.

Currently, FABRIC has two sets of facility ports connected to AmLight:

* Layer 2/VLAN facility ports
* Layer 3/IP facility ports

To use a Layer 2 facility port, experimenters must use of the following VLANs:

* 4000 to 4019

when using Layer 2 facility ports, experimenters are free to choose the IP addresses used by the hosts on both sides of the Layer 2 service.

To use a Layer 3 facility port, experimenters must use of the following VLANs:

* 3001 to 3008

When using Layer 3 facility ports, experimenters must use the IP prefixes predefined below, where the first IP and IPv6 of each range is a router:

* 3001 — 67.17.206.64/29, 2800:bc0:9000:3001::/64
* 3002 — 67.17.206.72/29, 2800:bc0:9000:3002::/64
* 3003 — 67.17.206.80/29, 2800:bc0:9000:3003::/64
* 3004 — 67.17.206.88/29, 2800:bc0:9000:3004::/64
* 3005 — 67.17.206.96/29, 2800:bc0:9000:3005::/64
* 3006 — 67.17.206.104/29, 2800:bc0:9000:3006::/64
* 3007 — 67.17.206.112/29, 2800:bc0:9000:3007::/64
* 3008 — 67.17.206.120/29, 2800:bc0:9000:3008::/64

When to choose Layer 2 or Layer 3 facility ports? If the experimenter wishes to build a private tunnel all the way to the destination. That implies that all network devices in the path will be configured to support the chosen VLAN ID. To connect to AmLight's Academic IP network, and with it, all its users and instruments, without any control but fast access, Layer 3 facility ports are the ideal solution.

## Attention
To use NRP resources, follow the instructions at https://nationalresearchplatform.org/documentation/

## Let's follow FABRIC's documentation to create a slice. We will leverage FABRIC's notebooks and documentation for this step. 

## Import the FABlib Library

The following creates a single node with basic compute capabilities. You build a slice by creating a new slice and adding resources to the slice. After you build the slice, you must submit a request for the slice to be instantiated.   

By default, the submit function will block until the node is ready and will display the progress of your slice being built.

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

Read the config to confirm.

In [ ]:
fablib.show_config();

## Create the Experiment Slice

The following steps create a single node with basic compute capabilities. You build a slice by creating a new slice and adding resources to the slice. After you build the slice, you must submit a request for the slice to be instantiated.   

By default, the submit function will block until the node is ready and will display the progress of your slice being built.


In [ ]:
# Choose any name for your slice
slice_name = "Slice-AWSDX-test-33"

# Facility port: Let's provide the config for Layer 2 facility port
facility_port_site = 'FIU'
facility_port = 'AmLight-EXP-Layer2-FIU'

In [ ]:
# Check if a slice with the same already exists

try:
    # If exists, delete it to avoid conflicts
    slice = fablib.get_slice(name=slice_name)
    slice.show()
except:
    print(f'Slice {slice_name} not found. Resuming... ')
    

In [ ]:
# Let's choose the VLAN based on the facility ports availability. VLANs listed in the column "Allocated VLAN Range" are NOT available.

filter_function=lambda s: s['site_name'] == facility_port_site
fablib.list_facility_ports(filter_function=filter_function)

In [ ]:
# For instance, let's use VLAN 4016 since it is part of the VLAN Range and it is not allocated to anyone (as per Allocated VLAN Range)

facility_port_vlan = "4016"

## Create Slice

In [ ]:
#Let's create a slice
slice.delete()
slice.show()
slice = fablib.new_slice(name=slice_name)
[site] = fablib.get_random_sites(count=1)

In [ ]:
# Create a node
node = slice.add_node(name=f"Node1", cores=4, ram=16, site=site, image='default_rocky_9')
node_iface = node.add_component(model='NIC_Basic', name="nic1").get_interfaces()[0]

# Make the node ready for perfSonar via Docker
node.add_post_boot_upload_directory('node_tools','.')
node.add_post_boot_execute('sudo bash node_tools/host_tune.sh')
node.add_post_boot_execute('sudo bash node_tools/enable_docker.sh {{ _self_.image }} ')
node.add_post_boot_execute('sudo docker pull fabrictestbed/slice-vm-rocky8-multitool:0.0.2 ')

In [ ]:
# Add a facility port to the slice
facility_port = slice.add_facility_port(name=facility_port, site=facility_port_site, vlan=facility_port_vlan)
facility_port_interface = facility_port.get_interfaces()[0]

# Connect the facility port and the node port (FABRIC network instantiates the path using a L2 tunnel)
net = slice.add_l2network(name=f'net_facility_port', interfaces=[])
net.add_interface(node_iface)
net.add_interface(facility_port_interface)

In [ ]:
#Submit the Request
slice.submit()

## Next steps

At this point, we have our FABRIC slice and virtual machine ready. If we are connecting to NRP at AmLight, we need to know:

1. Where is NRP located at AmLight ? We need to know the AmLight's switch and port. We can use the SDX-Lib to find out.
2. Where is the FABRIC Facility port at AmLight? We can use the SDX-Lib to find out.
3. Which VLAN ID to use? Because AtlanticWave-SDX supports VLAN translation, it is acceptable to have a VLAN ID at NRP and a different VLAN ID at the Facility port. We can use the SDX-Lib to find out the VLAN ranges available.
4. With the three questions answered, we can instantiate a L2VPN on AtlanticWave connecting NRP cluster to FIU's Facility Port using the SDX-Lib
5. We can then instantiate a NRP pod using the VLAN provided
6. Then we run our experiments!

### Requirements: Since we will be using SDX-Lib from this point forward, let's make sure it is installed.

Let's install the sdx-lib, a Python library made to be as compatible with FABRIC as possible. 
More details about the SDX-Lib can be found at: https://github.com/atlanticwave-sdx/sdx-lib

In [ ]:
sdxlib_version = "0.33.0"
!pip install --no-cache-dir -i https://test.pypi.org/simple/ sdxlib=={sdxlib_version}

In [ ]:
# Let's load the SDX-Lib
from sdxlib.sdx_client import SDXClient
from sdxlib.sdx_exception import SDXException

# Some external functions just for beautification  - unnecessary for the newest SDX-lib release
! from sdx_support import print_sdx_ports, print_sdx_services

###  1. Where is NRP located at AtlanticWave/AmLight ?

To locate a resource at AmLight, we can use the SDX-Lib.

## Assign URL Environment Test or Production

In [ ]:
# Production Environment URL
sdx_url = "https://sdxapi.atlanticwave-sdx.ai/production"

In [ ]:
# Test Environment URL
sdx_url = "https://sdxapi.atlanticwave-sdx.ai/test"

## Instantiate SDX Client

In [ ]:
# Let's instantiate the SDX-Lib via SDXClient class passing the URL above
client=SDXClient(sdx_url)

## Get all available ports 

In [ ]:
client.get_available_ports(search=None)

## Search specific Available Ports by Entities

In [ ]:
client.get_available_ports(search="NRP")

# Just attribute 
nrp_port_id = print_sdx_ports(sdx_client.get_available_ports(format="json"), search="NRP - <KNIT10>", return_id_only=True)
print(nrp_port_id)

In [ ]:
# Then we assign it manually:
nrp_port_id = "urn:sdx:port:amlight.net:JAX-LUM-SW01:21"

In [ ]:
# Let's choose a VLAN
nrp_vlan = "4001"

###  2. Where is FABRIC's facility port located at AmLight ?

Let's use the SDX-Lib again.

In [ ]:
client.get_available_ports(search="FABRIC")

In [ ]:
fabric_port_id = "urn:sdx:port:amlight.net:MIA-MI1-SW17:7"
fabric_vlan = "4016" # Chosen above during the instantiation of the facility port

## 3. Which VLAN ID to use? 

Because AtlanticWave-SDX supports VLAN translation, it is acceptable to have a VLAN ID at NRP and a different VLAN ID at the Facility port. We identified the VLAN IDs to use in the previous steps.

## 4. Build a L2VPN at AmLight connecting NRP to FABRIC via Facility Port

In [ ]:
# There are two mandatory attributes to create a L2VPN/VLAN on AmLight:
# a name
# a list of endpoints, each endpoint with its port_id and a VLAN ID

l2vpn_name = "FABRIC KNIT 10 L2VPN between FABRIC and NRP"
l2vpn_endpoints = [
    {"port_id": fabric_port_id, "vlan": fabric_vlan}, 
    {"port_id": nrp_port_id, "vlan": nrp_vlan},
]

sdx_l2vpn = SDXClient(sdx_url, l2vpn_name, l2vpn_endpoints)

try:
    my_knit10_l2vpn = sdx_l2vpn.create_l2vpn()
    print(f"L2VPN response: {my_knit10_l2vpn.get('message')}")
except SDXException as e:
    print(f"L2VPN creation failed: {e}") 


# Get L2VPN Service Id filtered by L2vpn name

In [ ]:
service_id = sdx_l2vpn.get_all_l2vpns(format="json", search="FABRIC KNIT 10 L2VPN between FABRIC and NRP")[0].get('Service ID')

# Print L2VPN Attributes for the retrieved Service Id

In [ ]:
sdx_l2vpn.get_l2vpn(service_id=service_id)

## In case we need to delete the L2VPN

In [ ]:
sdx_l2vpn.delete_l2vpn(service_id=service_id)

# NRP Section: Instantiating a NRP K8s Pod

In this section, we will instantiate a pod at NRP and start an iPerf3 session. More documentation about NRP can be found at: https://nationalresearchplatform.org/documentation/

#### Step 1: upload your kube config to the kube_config folder. This step is manual and the file's content is sensitive. 

Follow steps in the NRP documentation.

#### Step 2: Download the Python's NRP wraper

https://github.com/atlanticwave-sdx/sdx-sc24-nautilus-demo/blob/master/nautilus_utils.py

#### Step 3: Install the Kubernetes libraries

In [ ]:
%pip install kubernetes

#### Step 4: Import the Python libraries

In [ ]:
from nautilus_utils import create_deployment, run_command, wait_for_pod_ready, delete_vlan, delete_deployment, reload_config

In [ ]:
# Let's create the deployment using the Python K8s library. This function will return True if everything goes well.

#from kubernetes import client
#app_v1 = client.AppsV1Api()
#app_v1.create_namespaced_deployment(namespace=NAMESPACE, body=deploy_dict)

In [ ]:
create_deployment(
    name="amlight-demo",
    image="gitlab-registry.nrp-nautilus.io/prp/perfsonar/testpoint",
    nrp_node="k8s-gen4-01.ampath.net",
    node_iface="enp193s0f1",
    vlan=3999
)

In [ ]:
# Let's wait for the pod to get ready. It will return True.

In [ ]:
wait_for_pod_ready(name="amlight-demo")

In [ ]:
# Let's run an ifconfig on our instance

In [ ]:
print(run_command("amlight-demo", "ip addr show dev net1"))

In [ ]:
# Let's start iperf3 

In [ ]:
print(run_command("amlight-demo", "iperf3 -sD"))

### Run the iperf3 tests between our FABRIC slice's virtual machine and the NRP pod's iperf3 server


In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

# Let's choose a random subnet but it needs to be coordinated with the NRP deployment
subnet = IPv4Network("10.1.11.0/24")
node_addr = list(subnet)[2]

node = slice.get_node(name=f"Node1")        
node_iface = node.get_interface(network_name=f'net_facility_port') 


In [ ]:
node_iface.ip_addr_add(addr=node_addr, subnet=subnet)

In [ ]:
stdout, stderr = node.execute(f'sudo ip link set dev {node_iface.get_physical_os_interface_name()} up')
stdout, stderr = node.execute(f'sudo ip link set dev {node_iface.get_os_interface()} up')
stdout, stderr = node.execute(f'ip addr show {node_iface.get_os_interface()}')

Replace the IP address below for the IP address provided by NRP's ifconfig execution.

In [ ]:
nrp_ip = '10.1.11.15'
stdout, stderr = node.execute(f'ping -c 5 {nrp_ip}')

In [ ]:
stdout, stderr = node.execute("docker run --rm "
                                "--network host "
                                "fabrictestbed/slice-vm-rocky8-multitool:0.0.2 "
                                f"iperf3 -c {nrp_ip} -w 512k -i 1 -n 50M"
                                , quiet=False, output_file=f"{node.get_name()}.log");


## Cleaning up before we go!

In [ ]:
# Delete the SDX L2VPN created
#sdx_l2vpn.delete_l2vpn(service_id=service_id)

# Delete the FABRIC slice
#slice.delete()

In [ ]:
print("Thanks! See you next time!")